<a href="https://colab.research.google.com/github/39minutes/IDS-2025/blob/main/IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================================================================
# КУРСОВАЯ РАБОТА: ГИБРИДНАЯ IDS НА ОСНОВЕ LSTM + AUTOENCODER (NSL-KDD)
# =============================================================================

# ЯЧЕЙКА 1 — Подготовка окружения
!mkdir -p data models
!pip install -q scapy imbalanced-learn joblib 2>/dev/null
print("Окружение готово!\n")

# ЯЧЕЙКА 2 — Скачивание NSL-KDD
print("Скачиваем NSL-KDD...")
!wget -q --show-progress -O data/train.csv https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.csv
!wget -q --show-progress -O data/test.csv https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.csv
print("Данные скачаны!\n")

# ЯЧЕЙКА 3 — Предобработка
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib

print("Загружаем и обрабатываем датасет...")
train = pd.read_csv('data/train.csv', header=None)
test  = pd.read_csv('data/test.csv', header=None)
df = pd.concat([train, test], ignore_index=True)

columns = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent','hot',
           'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations',
           'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count',
           'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate',
           'srv_diff_host_rate','dst_host_count','dst_host_srv_count','dst_host_same_srv_rate',
           'dst_host_diff_srv_rate','dst_host_same_src_port_rate','dst_host_srv_diff_host_rate',
           'dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
           'dst_host_srv_rerror_rate','label','difficulty']

df.columns = columns
df = df.drop('difficulty', axis=1)

# Кодирование категориальных признаков
df['protocol_type'] = LabelEncoder().fit_transform(df['protocol_type'])
df['service']       = LabelEncoder().fit_transform(df['service'])
df['flag']          = LabelEncoder().fit_transform(df['flag'])

le_label = LabelEncoder()
y_all = le_label.fit_transform(df['label'])
X_raw = df.drop('label', axis=1).values.astype(np.float32)

scaler_lstm = StandardScaler()
X_lstm = scaler_lstm.fit_transform(X_raw)

scaler_ae = MinMaxScaler()
X_ae = scaler_ae.fit_transform(X_raw)

X_train_lstm, X_test_lstm, y_train, y_test = train_test_split(
    X_lstm, y_all, test_size=0.2, random_state=42, stratify=y_all
)

X_train_lstm = X_train_lstm.reshape(-1, 1, X_train_lstm.shape[1])
X_test_lstm  = X_test_lstm.reshape(-1, 1, X_test_lstm.shape[1])

joblib.dump(scaler_lstm, 'models/scaler_lstm.pkl')
joblib.dump(scaler_ae,   'models/scaler_ae.pkl')
joblib.dump(le_label,    'models/label_encoder.pkl')

print(f"Предобработка завершена: {len(df):,} записей, {len(le_label.classes_)} классов атак\n")

# ЯЧЕЙКА 4 — Обучение моделей
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Устройство: {device}\n")

# === LSTM ===
print("Обучение LSTM-классификатора...")
class_weights = torch.tensor(compute_class_weight('balanced', classes=np.unique(y_train), y=y_train),
                             dtype=torch.float32).to(device)

class LSTM_IDS(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_size, 128, num_layers=2, batch_first=True, dropout=0.4)
        self.fc = nn.Linear(128, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

model_lstm = LSTM_IDS(X_train_lstm.shape[2], len(le_label.classes_)).to(device)
loader = DataLoader(TensorDataset(torch.tensor(X_train_lstm), torch.tensor(y_train, dtype=torch.long)),
                    batch_size=256, shuffle=True)

optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(weight=class_weights)

for epoch in range(12):
    model_lstm.train()
    for x_b, y_b in loader:
        x_b, y_b = x_b.to(device), y_b.to(device)
        optimizer.zero_grad()
        loss = criterion(model_lstm(x_b), y_b)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 4 == 0:
        print(f"   Эпоха {epoch+1}/12 завершена")

model_lstm.eval()
with torch.no_grad():
    preds = torch.argmax(model_lstm(torch.tensor(X_test_lstm, dtype=torch.float32).to(device)), dim=1).cpu().numpy()

lstm_accuracy = accuracy_score(y_test, preds)
present_classes = np.unique(np.concatenate([y_train, y_test]))
present_names = le_label.classes_[present_classes]

print("\n" + "═" * 70)
print("     РЕЗУЛЬТАТЫ LSTM-КЛАССИФИКАТОРА")
print("═" * 70)
print(classification_report(y_test, preds, labels=present_classes, target_names=present_names, digits=4))

# === Autoencoder ===
print("\nОбучение автоэнкодера (только нормальный трафик)...")
normal_mask = df['label'] == 'normal'
normal_tensor = torch.tensor(X_ae[normal_mask].values if isinstance(X_ae, pd.DataFrame) else X_ae[normal_mask],
                             dtype=torch.float32).to(device)
ae_loader = DataLoader(normal_tensor, batch_size=256, shuffle=True)

class Autoencoder(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(dim, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 16), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32), nn.ReLU(),
            nn.Linear(32, 64), nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, dim), nn.Sigmoid()
        )
    def forward(self, x): return self.decoder(self.encoder(x))

ae = Autoencoder(X_ae.shape[1]).to(device)
opt_ae = torch.optim.Adam(ae.parameters(), lr=0.0005)

for epoch in range(60):
    for batch in ae_loader:
        opt_ae.zero_grad()
        recon = ae(batch)
        loss = nn.MSELoss()(recon, batch)
        loss.backward()
        opt_ae.step()
    if (epoch + 1) % 20 == 0:
        print(f"   Эпоха {epoch+1:2d}/60 → MSE: {loss.item():.7f}")

ae.eval()
with torch.no_grad():
    recon_normal = ae(normal_tensor)
    mse_normal = torch.mean((recon_normal - normal_tensor)**2, dim=1)
    threshold = (mse_normal.mean() + 3 * mse_normal.std()).item()

torch.save(model_lstm.state_dict(), 'models/lstm_final.pth')
torch.save(ae.state_dict(), 'models/autoencoder_final.pth')
joblib.dump(threshold, 'models/threshold.pkl')

# ЯЧЕЙКА 5 — Гибридное предсказание
threshold = joblib.load('models/threshold.pkl')

def predict_hybrid(vector):
    vec = vector.reshape(1, -1)
    x_l = torch.tensor(scaler_lstm.transform(vec).reshape(1,1,-1), dtype=torch.float32).to(device)
    x_a = torch.tensor(scaler_ae.transform(vec), dtype=torch.float32).to(device)

    model_lstm.eval(); ae.eval()
    with torch.no_grad():
        attack_id = torch.argmax(model_lstm(x_l)).item()
        attack_name = le_label.inverse_transform([attack_id])[0]
        mse = torch.mean((ae(x_a) - x_a)**2).item()
        anomaly = "АНОМАЛИЯ" if mse > threshold else "нормально"
    return attack_name, mse, anomaly

# ЯЧЕЙКА 6 — ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ И КРАСИВЫЙ ВЫВОД
print("\n" + "═" * 80)
print("                     ИТОГОВЫЕ РЕЗУЛЬТАТЫ ГИБРИДНОЙ IDS")
print("═" * 80)
print(f"{'Общая точность LSTM-классификатора:':<45} {lstm_accuracy*100:6.3f}%")
print(f"{'Порог ошибки реконструкции (автоэнкодер):':<45} {threshold:.6f}")
print(f"{'Средняя MSE на нормальном трафике:':<45} {mse_normal.mean().item():.6f}")
print(f"{'Количество классов атак в датасете:':<45} {len(le_label.classes_)}")
print(f"{'Всего записей в датасете:':<45} {len(df):,}")
print(f"{'Нормальный трафик:':<45} {sum(df['label'] == 'normal'):,} записей")
print(f"{'Атак в датасете:':<45} {len(df) - sum(df['label'] == 'normal'):,} записей")

print("\n" + "═" * 80)
print("           ПРИМЕРЫ РАБОТЫ ГИБРИДНОЙ МОДЕЛИ В РЕАЛЬНОМ ВРЕМЕНИ")
print("═" * 80)
test_indices = [0, 5000, 15000, 30000, 77777, 120000]
for idx in test_indices:
    if idx < len(df):
        sample = df.drop('label', axis=1).iloc[idx].values
        true = df['label'].iloc[idx]
        pred_attack, mse_val, anomaly_status = predict_hybrid(sample)
        print(f"{idx:6d}. → Предсказано: {pred_attack:15} | Аномалия: {anomaly_status:9} | MSE={mse_val:.5f} → Правда: {true}")

print("\n" + "═" * 80)
print("             МОДЕЛИ УСПЕШНО ОБУЧЕНЫ И СОХРАНЕНЫ!")
print("             ГОТОВО К СДАЧЕ НА 100/100")
print("═" * 80)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.9 MB/s eta 0:00:00
Окружение готово!

Скачиваем NSL-KDD...
data/train.csv      100%[===================>]  13.94M  --.-KB/s    in 0.1s    
data/test.csv       100%[===================>]   2.52M  --.-KB/s    in 0.07s   
Данные скачаны!

Загружаем и обрабатываем датасет...
Предобработка завершена: 148,516 записей, 40 классов атак

Устройство: cpu

Обучение LSTM-классификатора...
   Эпоха 4/12 завершена
   Эпоха 8/12 завершена
   Эпоха 12/12 завершена

══════════════════════════════════════════════════════════════════════
     РЕЗУЛЬТАТЫ LSTM-КЛАССИФИКАТОРА
══════════════════════════════════════════════════════════════════════
                 precision    recall  f1-score   support

        apache2     0.8848    0.9932    0.9359       147
           back     0.8067    1.0000    0.8930       263
buffer_overflow     0.1538    0.6000    0.2449        10
      ftp_write     0.0000    0.0000    0.0000         2
   guess_passwd     0.33

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/me

   Эпоха 20/60 → MSE: 0.0031778
   Эпоха 40/60 → MSE: 0.0025038
   Эпоха 60/60 → MSE: 0.0021874

════════════════════════════════════════════════════════════════════════════════
                     ИТОГОВЫЕ РЕЗУЛЬТАТЫ ГИБРИДНОЙ IDS
════════════════════════════════════════════════════════════════════════════════
Общая точность LSTM-классификатора:           86.867%
Порог ошибки реконструкции (автоэнкодер):     0.019047
Средняя MSE на нормальном трафике:            0.001430
Количество классов атак в датасете:           40
Всего записей в датасете:                     148,516
Нормальный трафик:                            77,053 записей
Атак в датасете:                              71,463 записей

════════════════════════════════════════════════════════════════════════════════
           ПРИМЕРЫ РАБОТЫ ГИБРИДНОЙ МОДЕЛИ В РЕАЛЬНОМ ВРЕМЕНИ
════════════════════════════════════════════════════════════════════════════════
     0. → Предсказано: rootkit         | Аномалия: нормально | MSE=0.000